### Dimension Table for Product

In [0]:
from pyspark.sql.functions import *
from delta.tables import DeltaTable

In [0]:
load_type = 'full'
if load_type == 'incremental':
  #Get the latest date from the silver table
  latest_date = spark.sql("select max(ingestion_date) from inventory_project.silver.erp_products_silver").collect()[0][0]
  print(latest_date)
  source_df = spark.read.table('inventory_project.silver.erp_products_silver')\
      .filter(col('ingestion_date') == latest_date)
else:
  source_df = spark.read.table('inventory_project.silver.erp_products_silver')

In [0]:
source_df = source_df.select("product_id","sku_code","brand","category","cost","status")
### Generate surrogate key
source_df = source_df.withColumn("product_key", concat(lit('prod'),lit('_'),col('product_id')))
source_df = source_df.withColumn("dim_updated_date", current_date())
if load_type == 'incremental':
    target_table = DeltaTable.forName(spark, "inventory_project.gold.erp_product_dim")
    target_table.alias("t")\
        .merge(source_df.alias("s"), "t.product_key = s.product_key")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    source_df.write.mode("overwrite").saveAsTable("inventory_project.gold.erp_product_dim")